In [1]:
from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster(threads_per_worker=1)

In [2]:
from dask.distributed import Client, wait
import time

import dask
import dask_cudf
import dask.dataframe as dd

import pandas as pd

import cudf
import numpy as np

import pandas.testing

from dask_cuml import knn as cumlKNN


In [3]:
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:34034 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 50.39 GB


In [4]:
def create_df(f, m, n):
    X = np.random.rand(m, n)
    ret = cudf.DataFrame([(i,X[:,i].astype(np.float32)) for i in range(n)])
    return ret

def get_meta(df):
    return df.iloc[:0]

In [5]:
workers = client.has_what().keys()
workers

dict_keys(['tcp://127.0.0.1:33596', 'tcp://127.0.0.1:33890'])

In [6]:
# Per gpu/worker
train_m = 500 
train_n = 1000

In [7]:
search_m = 50
search_k = 15

In [8]:
%%time

# Create dfs on each worker (gpu)
dfs = [client.submit(create_df, n, train_m, train_n, workers = [worker])
       for worker, n in list(zip(workers, range(len(cluster.workers))))]

# Wait for completion
wait(dfs)

meta = client.submit(get_meta, dfs[0]).result()

CPU times: user 580 ms, sys: 194 ms, total: 774 ms
Wall time: 6.13 s


In [9]:
lr = cumlKNN.KNN()

In [10]:
X_df = dask_cudf.from_delayed(dfs, meta=meta)

In [11]:
client.who_has()

{'create_df-d39b74d48bef5527496eedfe9a5cc87e': ('tcp://127.0.0.1:33596',),
 'create_df-583622552a88c16b167fe01783afdda9': ('tcp://127.0.0.1:33890',)}

In [12]:
%%time
lr.fit(X_df)

CPU times: user 338 ms, sys: 19.3 ms, total: 357 ms
Wall time: 1.92 s


In [13]:
%%time
lr.kneighbors(X_df[0:search_m], search_k)

CPU times: user 1.78 s, sys: 129 ms, total: 1.91 s
Wall time: 3.42 s


(<dask_cudf.DataFrame | 4 tasks | 2 npartitions>,
 <dask_cudf.DataFrame | 4 tasks | 2 npartitions>)